In [55]:
import requests
import json
import pandas as pd
import pgeocode ## https://pypi.org/project/pgeocode/

## Helper Functions

In [24]:
## For printing dictionaries nicely instead of using pretty print
def pretty_print(d, multiplier=1):
    if hasattr(d, 'keys'):
        for k in d.keys():
            print("  "*multiplier + k)
            pretty_print(d[k], multiplier+1)
    else:
        print("  "*multiplier + d)

In [27]:
## To set up the contnt from dataLink and prepare for mongodb insertMany
def createDictFromContent(contentList):
    legend = contentList[0]
    results = []
    
    for line in contentList[1:]:
        date,county,state,fips,cases,deaths = line.split(",")
        results.append({"date":   date, 
                       "county": county,
                       "state":  state,
                       "fips":   fips,
                       "cases":  cases,
                       "deaths": deaths})
    return results

In [115]:
## Converts fip to zip and returns data neecessary
## Includes zip by 
## 1. converting FIP to ZIPcode 
## 2. finding the lat,long coordinate
## 3. Including it as a datapoint
def combineAndFindCoordinate(listOfContentDict, zipMap):
    copied_listOfContentDict = listOfContentDict.copy()
    nomi = pgeocode.Nominatim('us')
    for d in copied_listOfContentDict:
        print("\n\n")
        fips = d["fips"]
        print(d)
        zipcodesInFIP = [z for z in zipMap[zipMap['STCOUNTYFP']==fips]["ZIP"]]
        print("All zipcodese in FIP:")
        print(zipcodesInFIP)
        nomiResults = map(nomi.query_postal_code, zipcodesInFIP)
        print("After nomi results")
        print(zipcodesInFIP)
        pairsPerZip = [(n["latitude"], n["longitude"]) for n in nomiResults]
        d["county_zips_latitudes"] = [pairsPerZip[0] for pair in pairsPerZip]
        d["county_zips_longitudes"] = [pairsPerZip[1] for pair in pairsPerZip]
    return copied_listOfContentDict

In [ ]:
def extract_necessary_content

## Parsing the Data

##### Data from Daily Updated Covid Cases
Uses the https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv  updated daily

In [28]:
dataLink = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
r = requests.get(dataLink)


In [29]:
content = r.content
# dataLinkAsDict = json.loads(content.decode('utf-8'))
rawContent = content.decode('utf-8')
rawContentSplitByLine = rawContent.split("\n")

contentForMongo = createDictFromContent(rawContentSplitByLine)

In [30]:
contentForMongo[:5]

[{'date': '2020-01-21',
  'county': 'Snohomish',
  'state': 'Washington',
  'fips': '53061',
  'cases': '1',
  'deaths': '0'},
 {'date': '2020-01-22',
  'county': 'Snohomish',
  'state': 'Washington',
  'fips': '53061',
  'cases': '1',
  'deaths': '0'},
 {'date': '2020-01-23',
  'county': 'Snohomish',
  'state': 'Washington',
  'fips': '53061',
  'cases': '1',
  'deaths': '0'},
 {'date': '2020-01-24',
  'county': 'Cook',
  'state': 'Illinois',
  'fips': '17031',
  'cases': '1',
  'deaths': '0'},
 {'date': '2020-01-24',
  'county': 'Snohomish',
  'state': 'Washington',
  'fips': '53061',
  'cases': '1',
  'deaths': '0'}]

##### Data for FIPS to Long, Lat pairs
Uss data from data.world at https://data.world/niccolley/us-zipcode-to-county-state. Downloaded and placed in here:

https://data.world/mgecmen/covid-19/workspace/file?filename=US_COUNTY_DAILY_UPDATE.csv

Create a FIPS:Long,Lat Mapping
1. Get FIP to Zipcode mapping from local `fips_zip_metadata.csv`
2. Use `pgeocode` to convert zipcode to longitud, latitude and create mapping

In [62]:
!ls

DataHandling.ipynb          fips_zip_metadata.csv
ZIP-COUNTY-FIPS_2010-03.csv


In [109]:
zips_df = pd.read_csv('fips_zip_metadata.csv')
display(zips_df.head())


zips_census_df =  pd.read_csv('ZIP-COUNTY-FIPS_2010-03.csv')
display(zips_census_df.head())
len(zips_df), len(zips_census_df)

,zip,fips
0,36008,1001
1,36532,1003
2,36580,1003
3,36547,1003
4,36577,1003


,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP
0,36003,Autauga County,AL,1001,H1
1,36006,Autauga County,AL,1001,H1
2,36008,Autauga County,AL,1001,H1
3,36022,Autauga County,AL,1001,H1
4,36051,Autauga County,AL,1001,H1


(53962, 46855)

In [81]:
zipcode = zips_df[zips_df['fips']==56037]
[z for z in zipcode["zip"]]

[82322, 82935, 82945, 82932, 82934, 82942, 82929, 82943, 82901, 82336, 82902]

In [40]:
dfConvertedToDict = df.to_dict()

In [45]:
dfConvertedToDict.keys()

dict_keys(['FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Last_Update', 'Lat', 'Long_', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Combined_Key'])

In [116]:
combinedResult = combineAndFindCoordinate(contentForMongo[:5], zips_census_df)




{'date': '2020-01-21', 'county': 'Snohomish', 'state': 'Washington', 'fips': '53061', 'cases': '1', 'deaths': '0', 'county_zips_latitudes': [], 'county_zips_longitudes': []}
All zipcodese in FIP:
[]
After nomi results
[]



{'date': '2020-01-22', 'county': 'Snohomish', 'state': 'Washington', 'fips': '53061', 'cases': '1', 'deaths': '0', 'county_zips_latitudes': [], 'county_zips_longitudes': []}
All zipcodese in FIP:
[]
After nomi results
[]



{'date': '2020-01-23', 'county': 'Snohomish', 'state': 'Washington', 'fips': '53061', 'cases': '1', 'deaths': '0', 'county_zips_latitudes': [], 'county_zips_longitudes': []}
All zipcodese in FIP:
[]
After nomi results
[]



{'date': '2020-01-24', 'county': 'Cook', 'state': 'Illinois', 'fips': '17031', 'cases': '1', 'deaths': '0', 'county_zips_latitudes': [], 'county_zips_longitudes': []}
All zipcodese in FIP:
[]
After nomi results
[]



{'date': '2020-01-24', 'county': 'Snohomish', 'state': 'Washington', 'fips': '53061', 'cases': '1', 'deaths'

In [118]:
display(zips_df[zips_df["fips"]=="53061"])
display(zips_census_df[zips_census_df["STCOUNTYFP"]=="53061"])

,zip,fips


,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP


In [96]:
combinedResult

[{'date': '2020-01-21',
  'county': 'Snohomish',
  'state': 'Washington',
  'fips': '53061',
  'cases': '1',
  'deaths': '0',
  'county_zips_latitudes': [],
  'county_zips_longitudes': []},
 {'date': '2020-01-22',
  'county': 'Snohomish',
  'state': 'Washington',
  'fips': '53061',
  'cases': '1',
  'deaths': '0',
  'county_zips_latitudes': [],
  'county_zips_longitudes': []},
 {'date': '2020-01-23',
  'county': 'Snohomish',
  'state': 'Washington',
  'fips': '53061',
  'cases': '1',
  'deaths': '0',
  'county_zips_latitudes': [],
  'county_zips_longitudes': []},
 {'date': '2020-01-24',
  'county': 'Cook',
  'state': 'Illinois',
  'fips': '17031',
  'cases': '1',
  'deaths': '0',
  'county_zips_latitudes': [],
  'county_zips_longitudes': []},
 {'date': '2020-01-24',
  'county': 'Snohomish',
  'state': 'Washington',
  'fips': '53061',
  'cases': '1',
  'deaths': '0',
  'county_zips_latitudes': [],
  'county_zips_longitudes': []},
 {'date': '2020-01-25',
  'county': 'Orange',
  'state': 